# 0.0. Mission

## Context

## About the company

Instacart is an American company that operates a grocery delivery and pick-up service in the United <br>
States and Canada. The enterprise has a website where customers can select products to buy. 

## Challenge

**INCREASE THE REVENUE**

### Ideas 

- Make the current customers buy more products and with more frequency.
- Find new customers

We will focus on the first idea, because our dataset only contains data about the time and items <br>
purchased by each customer, to aproach the second idea it would be necessary to have customer personal<br> information as age, address and gender for example.

### Selected Idea

**Make the current customers buy more products and with more frequency.**

### HOW ?

One approach: **Segmentating the customers**

Marketing and Sales 

So it will be possible to:

-Select the main products each group buys more oftenly, and recommend similar products to them.

-Offer promotions to the most loyalty group.

-Create incentives to infrequent or new customers. 

-Create marketing strategies to each group. 



### RFM - Analysis for Customer Segmentation

We are going to use RFM Analysis, that focus on Recency, Frequency and Monetary information about customers

As we dont have information about recency and monetary in our dataset, we are going <br>
to consider the average recency from previous orders and consider the amount of <br>
products as the monetary information.

# 1.0. Preparing Process

## 1.1. IMPORTS

In [79]:
import pandas as pd
import math 
import numpy as np
from scipy import stats as ss 

###--------- PANDAS - EXIBIR TODAS COLUNAS ----###

pd.set_option('display.max_columns', None)


###--------- ESTILIZAÇÃO DO NOTEBOOK ---------###

from IPython.core.display import display, HTML

# retira a margem do notebook
display(HTML("<style>.container { width:100% !important; }</style>"))

# font do texto markdown
display(HTML("<style>h1 { font-size:23px !important; }</style>"))
display(HTML("<style>h2 { font-size:20px !important; }</style>"))
display(HTML("<style>h3 { font-size:17px !important; }</style>"))
display(HTML("<style>h4 { font-size:16px !important; }</style>"))
display(HTML("<style>p { font-size:16px !important; }</style>"))

# tamanho da fonte da tabela
display(HTML("<style>th { font-size:15px !important; }</style>"))
display(HTML("<style>td { font-size:15px !important; }</style>"))

# font do codigo 
display(HTML("<style>span { font-size:16px !important; }</style>"))

## 1.2. Helper Functions

## 1.3. Merging data

In [2]:
'''
aisles_df = pd.read_csv('data/aisles.csv')
departments_df = pd.read_csv('data/departments.csv')
order_products_prior_df = pd.read_csv('data/order_products__prior.csv')
order_products_train_df = pd.read_csv('data/order_products__train.csv')
orders_df = pd.read_csv('data/orders.csv')
products_df = pd.read_csv('data/products.csv')
'''

"\naisles_df = pd.read_csv('data/aisles.csv')\ndepartments_df = pd.read_csv('data/departments.csv')\norder_products_prior_df = pd.read_csv('data/order_products__prior.csv')\norder_products_train_df = pd.read_csv('data/order_products__train.csv')\norders_df = pd.read_csv('data/orders.csv')\nproducts_df = pd.read_csv('data/products.csv')\n"

In [3]:
# Dimensions
'''
print( "aisles shape: " + str(aisles_df.shape) )
print( "departments shape: " + str(departments_df.shape) )
print( "order_products_prior shape: " + str(order_products_prior_df.shape) )
print( "order_products_train shape: " + str(order_products_train_df.shape) )
print( "orders shape: " + str(orders_df.shape) )
print( "products shape: " + str(products_df.shape) )
'''

'\nprint( "aisles shape: " + str(aisles_df.shape) )\nprint( "departments shape: " + str(departments_df.shape) )\nprint( "order_products_prior shape: " + str(order_products_prior_df.shape) )\nprint( "order_products_train shape: " + str(order_products_train_df.shape) )\nprint( "orders shape: " + str(orders_df.shape) )\nprint( "products shape: " + str(products_df.shape) )\n'

### order_products_prior

In [4]:
# order_products_prior_df.head(3)

### order_products_train

In [5]:
#order_products_train_df.head(3)

### Append the order_products_train to the end of order_products_prior

In [6]:
#order_products_df = order_products_prior_df.append(order_products_train_df, ignore_index=True)

In [7]:
#order_products_df.head(3)

### orders

In [8]:
#orders_df.head(3)

### products

In [9]:
#products_df.head(3)

### Aisles

In [10]:
#aisles_df.head(3)

### departments

In [11]:
#departments_df.head(3)

### JOIN the tables: products, Aisles and departments

In [12]:
# Join the three tables: products, aisles, departments
#products_info_df = pd.merge(products_df, aisles_df, how='inner', on='aisle_id')
#products_info_df = pd.merge(products_info_df, departments_df, how='inner', on='department_id')

# rearrange columns
#columns_names = list( products_info_df.columns )
#columns_names[3], columns_names[4] = columns_names[4], columns_names[3]
#products_info_df = products_info_df.reindex(columns=columns_names)

In [13]:
#products_info_df.head(5)

### JOIN orders, order_products and products info 

In [14]:
#df1 = pd.merge(order_products_df, orders_df, how='inner', on='order_id')
#df1 = pd.merge(df1, products_info_df, how='inner', on='product_id')

# sort the order_id column
#df1.sort_values(by='order_id', inplace=True)

# save the new dataset 
#df1 = df1.iloc[:1000000, :]
#df1.to_csv(path_or_buf='data/dataset.csv')

## 1.4. Loading Data

In [15]:
df_raw = pd.read_csv('data/dataset.csv')

In [16]:
df_raw.drop(axis=0, columns=['Unnamed: 0','eval_set'], inplace=True)

In [17]:
df_raw.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name,aisle_id,aisle,department_id,department
0,1,43633,5,1,112108,4,4,10,9.0,Lightly Smoked Sardines in Olive Oil,95,canned meat seafood,15,canned goods
1,1,47209,7,0,112108,4,4,10,9.0,Organic Hass Avocado,24,fresh fruits,4,produce
2,1,10246,3,0,112108,4,4,10,9.0,Organic Celery Hearts,83,fresh vegetables,4,produce
3,1,49302,1,1,112108,4,4,10,9.0,Bulgarian Yogurt,120,yogurt,16,dairy eggs
4,1,13176,6,0,112108,4,4,10,9.0,Bag of Organic Bananas,24,fresh fruits,4,produce


In [18]:
df_raw.shape

(1000000, 14)

# 2.0. Data Description

In [19]:
df2 = df_raw.copy()

## 2.1. Rename columns

In [20]:
df2.columns

Index(['order_id', 'product_id', 'add_to_cart_order', 'reordered', 'user_id',
       'order_number', 'order_dow', 'order_hour_of_day',
       'days_since_prior_order', 'product_name', 'aisle_id', 'aisle',
       'department_id', 'department'],
      dtype='object')

In [21]:
cols_old_name = [
                'order_id', 'product_id', 'add_to_cart_order', 'reordered', 'user_id',
                'order_number', 'order_dow', 'order_hour_of_day',
                'days_since_prior_order', 'product_name', 'aisle_id', 'aisle',
                'department_id', 'department'
]

# changed order_dow to order_day_of_week
cols_new_name = [
                'order_id', 'product_id', 'add_to_cart_order', 'reordered', 'user_id',
                'order_number', 'order_day_of_week', 'order_hour_of_day',
                'days_since_prior_order', 'product_name', 'aisle_id', 'aisle',
                'department_id', 'department'
]

In [22]:
df2.columns = cols_new_name

## 2.2. Data Dimensions

In [23]:
print('Number of rows: {}'.format( df2.shape[0] ))
print('Number of columns: {}'.format( df2.shape[1] ))

Number of rows: 1000000
Number of columns: 14


## 2.3. Data Types

In [24]:
df2.dtypes

order_id                    int64
product_id                  int64
add_to_cart_order           int64
reordered                   int64
user_id                     int64
order_number                int64
order_day_of_week           int64
order_hour_of_day           int64
days_since_prior_order    float64
product_name               object
aisle_id                    int64
aisle                      object
department_id               int64
department                 object
dtype: object

## 2.4. Check NA

In [35]:
df2.isna().sum()

order_id                      0
product_id                    0
add_to_cart_order             0
reordered                     0
user_id                       0
order_number                  0
order_day_of_week             0
order_hour_of_day             0
days_since_prior_order    61293
product_name                  0
aisle_id                      0
aisle                         0
department_id                 0
department                    0
dtype: int64

## 2.5. Fillout NA

days_since_prior_order - a NA means that the customer is a new customer, that has never bought anything previously, so we will change the Nan by a -1.

In [40]:
df2['days_since_prior_order'] = df2['days_since_prior_order'].apply(lambda x: -1 if math.isnan( x ) else x )

## 2.6. Change types

In [46]:
df2.dtypes

order_id                    int64
product_id                  int64
add_to_cart_order           int64
reordered                   int64
user_id                     int64
order_number                int64
order_day_of_week           int64
order_hour_of_day           int64
days_since_prior_order    float64
product_name               object
aisle_id                    int64
aisle                      object
department_id               int64
department                 object
dtype: object

In [49]:
# days_since_prior_order can be converted from float to int
df2['days_since_prior_order'] = df2['days_since_prior_order'].astype(int)

In [50]:
df2.dtypes

order_id                   int64
product_id                 int64
add_to_cart_order          int64
reordered                  int64
user_id                    int64
order_number               int64
order_day_of_week          int64
order_hour_of_day          int64
days_since_prior_order     int64
product_name              object
aisle_id                   int64
aisle                     object
department_id              int64
department                object
dtype: object

## 2.7. Descriptive Statistics

### Split data between numerical and categorical variables

In [52]:
num_data = df2.select_dtypes( include=['int64'] )
cat_data = df2.select_dtypes( exclude=['int64'] )

### Statistics

In [100]:
min_df = pd.DataFrame( num_data.apply(lambda x: min(x), axis=0) ).T
min_df['days_since_prior_order'] = min( num_data[ num_data['days_since_prior_order'] != -1 ]['days_since_prior_order'] )

max_df = pd.DataFrame( num_data.apply(lambda x : max(x), axis=0) ).T

range_df = pd.DataFrame( num_data.apply(lambda x : max(x) - min(x), axis=0 )).T

mean_df = pd.DataFrame( num_data.apply(lambda x : np.mean(x), axis=0 )).T

median_df = pd.DataFrame( num_data.apply(lambda x : np.median(x), axis=0 )).T

std_df = pd.DataFrame( num_data.apply(lambda x : np.std(x), axis=0 )).T

skew_df = pd.DataFrame( num_data.apply(lambda x : x.skew(), axis=0 )).T

kurtosis_df = pd.DataFrame( num_data.apply(lambda x : x.kurtosis(), axis=0 )).T


attributes_table = pd.concat([min_df, max_df, range_df, mean_df, median_df, std_df, skew_df, kurtosis_df]).T.reset_index()
attributes_table.columns = ['attributes', 'min', 'max', 'range', 'mean', 'median', 'std', 'skew', 'kurtosis']
attributes_table

,attributes,min,max,range,mean,median,std,skew,kurtosis
0,order_id,1.0,101299.0,101298.0,50663.107511,50642.0,29287.149555,0.001838,-1.203221
1,product_id,1.0,49688.0,49687.0,25576.224010,25266.0,14098.905624,-0.021020,-1.141383
2,add_to_cart_order,1.0,127.0,126.0,8.356323,6.0,7.163926,1.928983,7.325248
3,reordered,0.0,1.0,1.0,0.590309,1.0,0.491777,-0.367277,-1.865111
4,user_id,4.0,206208.0,206204.0,102979.114973,102716.0,59435.539579,0.001047,-1.198345
5,order_number,1.0,100.0,99.0,17.321499,11.0,17.663996,1.755376,3.233232
6,order_day_of_week,0.0,6.0,6.0,2.742486,3.0,2.094194,0.180296,-1.335542
7,order_hour_of_day,0.0,23.0,23.0,13.423948,13.0,4.252317,-0.043954,-0.020598
8,days_since_prior_order,0.0,30.0,31.0,10.535240,7.0,9.092048,0.990787,-0.089505
9,aisle_id,1.0,134.0,133.0,71.202297,83.0,38.186122,-0.166665,-1.324655


In [125]:
# Obtaining the mode
mode_df = pd.DataFrame( num_data.apply(lambda x: ss.mode(x)) ).T.reset_index()
mode_df.columns = ['attributes', 'mode', 'frequency']
mode_df['mode'] = mode_df['mode'].apply(lambda x: x[0])
mode_df['frequency'] = mode_df['frequency'].apply(lambda x: x[0])

mode_df 

,attributes,mode,frequency
0,order_id,61355,127
1,product_id,24852,14681
2,add_to_cart_order,1,99106
3,reordered,1,590309
4,user_id,169550,260
5,order_number,1,61293
6,order_day_of_week,0,191985
7,order_hour_of_day,10,83554
8,days_since_prior_order,7,105817
9,aisle_id,24,112304


In [145]:
cat_data['product_name'].value_counts().sort_values(ascending=False).reset_index().rename(columns={'index':'product_name', 'product_name':'quantity'}).head(7)

,product_name,quantity
0,Banana,14681
1,Bag of Organic Bananas,11830
2,Organic Strawberries,8169
3,Organic Baby Spinach,7468
4,Organic Hass Avocado,6481
5,Organic Avocado,5381
6,Large Lemon,4752


In [146]:
cat_data['aisle'].value_counts().sort_values(ascending=False).reset_index().rename(columns={'index':'aisle', 'aisle':'quantity'}).head(7)

,aisle,quantity
0,fresh fruits,112304
1,fresh vegetables,105215
2,packaged vegetables fruits,54900
3,yogurt,44377
4,packaged cheese,30207
5,milk,27285
6,water seltzer sparkling water,25692


In [147]:
cat_data['department'].value_counts().sort_values(ascending=False).reset_index().rename(columns={'index':'department','department':'quantity'}).head(7)

,department,quantity
0,produce,292459
1,dairy eggs,166609
2,snacks,89212
3,beverages,83040
4,frozen,69089
5,pantry,57351
6,bakery,36300


# 3.0. Feature Engineering

## 3.1. Mind Map

## 3.2. Hypothesis creation

## 3.3. Final hypothesis list

## 3.4. Deriving new variables

# 4.0. Filtering variables

## 4.1. Filtering rows 

## 4.2. Selecting columns

# 5.0. Exploratory Data Analysis

## 5.1. Univariate Analysis

### 5.1.1.  Response Variable

### 5.1.2. Numerical Variables

### 5.1.3. Categorical Variables

## 5.2. Bivariate Analysis

### 5.2.1. HIP 1.

## 5.3. Multivariate Analysis

### 5.3.1. Numerical Attributes

### 5.3.2. Categorical Attributes

# 6.0. Data Preparation

## 6.1. Normalization

## 6.2. Rescaling 

## 6.3. Transformation

### 6.3.1. Encoding

### 6.3.2. Transformation of target variables

# 7.0. Feature Selection

## 7.1. Splitting between Train and Test set

## 7.2. Boruta as Automated Feature Selection

## 7.3. Manual Feature Selection

# 8.0. Machine Learning Model

# 9.0. Hyperparameter Fine Tuning

# 10.0. Performance analysis